In [ ]:
from IPython.core.magic import register_cell_magic
@register_cell_magic
def run_and_save(line, cell):
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

In [ ]:
!mkdir tecton-testing
%cd tecton-testing
!mkdir features
!mkdir tests
!tecton init
!tecton workspace create tecton-testing

In [ ]:
import tecton
tecton.login('community.tecton.ai')

In [ ]:
%%run_and_save ./features/transaction_amount_is_high.py

from tecton import RequestSource, on_demand_feature_view
from tecton.types import Field, Float64, Int64
import pandas

# Define the request schema
transaction_request = RequestSource(schema=[Field("amount", Float64)])

# Define the output schema
output_schema = [Field("transaction_amount_is_high", Int64)]

# This On-Demand Feature View evaluates a transaction amount and declares it as "high", if it's higher than 10000
@on_demand_feature_view(
    sources=[transaction_request],
    mode="pandas",
    schema=output_schema,
    description="Whether the transaction amount is considered high (over $10000)",
)
def transaction_amount_is_high(transaction_request: pandas.DataFrame):
    import pandas as pd

    df = pd.DataFrame()
    df["transaction_amount_is_high"] = (transaction_request["amount"] >= 10000).astype("int64")
    return df

In [ ]:
transaction_amount_is_high.validate()

transaction_request = pandas.DataFrame([{"amount": 100.0}])

result = transaction_amount_is_high.run(
    transaction_request=transaction_request,
)
print(result.to_pandas())

transaction_request = pandas.DataFrame([{"amount": 100000.0}])

result = transaction_amount_is_high.run(
    transaction_request=transaction_request,
)
print(result.to_pandas())

In [ ]:
%%run_and_save ./tests/transaction_amount_is_high.py
from features.transaction_amount_is_high import transaction_amount_is_high
import pandas

# Testing the 'transaction_amount_is_high' feature which depends on request data ('amount') as input
def test_transaction_amount_is_high():
    transaction_request = pandas.DataFrame({"amount": [124, 5001, 34235436234]})

    actual = transaction_amount_is_high.test_run(transaction_request=transaction_request)
    expected = pandas.DataFrame({"transaction_amount_is_high": [0, 1, 1]})

    pandas.testing.assert_frame_equal(actual, expected)

In [ ]:
test_transaction_amount_is_high()

In [ ]:
%%run_and_save ./features/transaction_amount_is_high.py

from tecton import RequestSource, on_demand_feature_view
from tecton.types import Field, Float64, Int64
import pandas

# Define the request schema
transaction_request = RequestSource(schema=[Field("amount", Float64)])

# Define the output schema
output_schema = [Field("transaction_amount_is_high", Int64)]

# This On-Demand Feature View evaluates a transaction amount and declares it as "high", if it's higher than r\5,000
@on_demand_feature_view(
    sources=[transaction_request],
    mode="pandas",
    schema=output_schema,
    description="Whether the transaction amount is considered high (over $5000)",
)
def transaction_amount_is_high(transaction_request: pandas.DataFrame):
    import pandas as pd

    df = pd.DataFrame()
    df["transaction_amount_is_high"] = (transaction_request["amount"] >= 5000).astype("int64")
    return df

In [ ]:
transaction_amount_is_high.validate()

In [ ]:
test_transaction_amount_is_high()

In [ ]:
!tecton test